In [1]:
import gc
import os
if not os.path.exists('../.hf_cache'):
    os.makedirs('../.hf_cache')
os.environ["HF_HOME"] = '../.hf_cache'

import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTConfig, SFTTrainer, setup_chat_format

/global/scratch/users/chenxin0210/conda-env/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = "meta-llama/Meta-Llama-3-8B"#"meta-llama/Meta-Llama-3-70B-Instruct"#


model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
print("Before:", tokenizer.special_tokens_map)
print("Vocab size before:", len(tokenizer))

model, tokenizer = setup_chat_format(model, tokenizer)
print("After:", tokenizer.special_tokens_map)
print("Vocab size after:", len(tokenizer))



In [3]:
adaptor_model = "./dft-llama-3b-lora-4gpu"
model = PeftModel.from_pretrained(model, adaptor_model)
model = model.merge_and_unload()

In [4]:
dataset = load_dataset("MaterialsAI/robocr_poscar")

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(base_model)
# row=[{"role": "system", "content": "instruction"}]
# tokenizer.apply_chat_template(row, tokenize=False)

In [6]:
# model, tokenizer = setup_chat_format(model, tokenizer)
# row=[{"role": "system", "content": "instruction"}]
# tokenizer.apply_chat_template(row, tokenize=False)

In [22]:
import re

def round_poscar_numbers(text, decimal_places=2):
    def round_number(match):
        return f"{float(match.group()):.{decimal_places}f}"

    # Pattern to match floating point numbers
    pattern = r'\d+\.\d+'

    # Round numbers in the text
    rounded_text = re.sub(pattern, round_number, text)

    
    # Remove leading/trailing whitespace from each line
    rounded_text = '\n'.join(' '.join(line.split()) for line in rounded_text.splitlines())

    return rounded_text

def format_chat_template(row):
    rounded_out = round_poscar_numbers(row["output"], decimal_places=3)
    row_json = [{"role": "system", "content": row["instruction"]}, {"role": "user", "content": row["input"]},
               {"role": "assistant", "content": rounded_out}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['train']['text'][13]

Map (num_proc=4): 100%|██████████| 137/137 [00:00<00:00, 206.36 examples/s]


'<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiBi(PO₄)₂ crystallizes in the triclinic P1 space group. There are three inequivalent Li¹⁺ sites. In the first Li¹⁺ site, Li¹⁺ is bonded in a distorted hexagonal planar geometry to six O²⁻ atoms. There are a spread of Li-O bond distances ranging from 2.47-2.58 Å. In the second Li¹⁺ site, Li¹⁺ is bonded in a 6-coordinate geometry to six O²⁻ atoms. There are three shorter (1.97 Å) and three longer (2.44 Å) Li-O bond lengths. In the third Li¹⁺ site, Li¹⁺ is bonded in a 6-coordinate geometry to six O²⁻ atoms. There are a spread of Li-O bond distances ranging from 1.97-2.46 Å. There are three inequivalent Bi⁵⁺ sites. In the first Bi⁵⁺ site, Bi⁵⁺ is bonded to six O²⁻ atoms to form BiO₆ octahedra that share corners with six PO₄ tetrahedra. There are three shorter (2.17 Å) and three longer (2.18 Å) Bi-O bond lengths. In the second Bi⁵⁺ site, Bi⁵⁺ is bonded in a 6-coordinate geometry to s

In [23]:
encoded = tokenizer.encode(dataset['test']['text'][0], add_special_tokens=True, max_length=None, truncation=True)
len(encoded)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


415

In [24]:
len(tokenizer.tokenize(dataset['train']['text'][4]))

2552

In [25]:
from transformers import pipeline

In [26]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    device_map='auto'
)

In [27]:
dataset['test']['text'][0]

'<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLi₂CaSiO₄ crystallizes in the tetragonal I̅42m space group. Li¹⁺ is bonded to four equivalent O²⁻ atoms to form distorted LiO₄ trigonal pyramids that share corners with four equivalent SiO₄ tetrahedra and corners with four equivalent LiO₄ trigonal pyramids. All Li-O bond lengths are 1.97 Å. Ca²⁺ is bonded in a 8-coordinate geometry to eight equivalent O²⁻ atoms. There are four shorter (2.39 Å) and four longer (2.76 Å) Ca-O bond lengths. Si⁴⁺ is bonded to four equivalent O²⁻ atoms to form SiO₄ tetrahedra that share corners with eight equivalent LiO₄ trigonal pyramids. All Si-O bond lengths are 1.66 Å. O²⁻ is bonded in a 5-coordinate geometry to two equivalent Li¹⁺, two equivalent Ca²⁺, and one Si⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nLi2 Ca1 Si1 O4\n1.000\n-2.525 2.525 3.322\n2.525 -2.525 3.322\n2.525 2.525 -3.322\nLi Ca Si O\n2 1 1 4\ndirect\n0.250 0.750 0.500 Li\n0.750 0.250

In [28]:
import re

def extract_input(text):
    # Find all content up to the last <|im_end|> before the assistant's response
    match = re.search(r'(.*<\|im_end\|>)\s*<\|im_start\|>assistant', text, re.DOTALL)
    if match:
        return match.group(1).strip() + '\n<|im_start|>assistant'
    return None

In [29]:
dataset['train']['text'][0]

'<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nLi4 Ga4 Se8 O24\n1.000\n

In [30]:
input_for_model = extract_input(dataset['train']['text'][0])
input_for_model

'<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant'

In [31]:
pipe(input_for_model, do_sample=False)

[{'generated_text': '<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nSe⁴⁺

In [20]:
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
[{'generated_text': '<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\nSe⁴⁺\n'}]

In [ ]:
[{'generated_text': '<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nLi1 Ga1 Se1 O6\n1.000\n0.000 0.000 0.000\n0.000 0.000 0.000\n0.000 0.000 0.000\nLi Ga Se O\n1 1 1 6\ndirect\n0.000 0.000 0.000 Li\n0.000 0.000 0.000 Ga\n0.000 0.000 0.000 Se\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000 O\n0.000 0.000 0.000'}]

In [ ]:
[{'generated_text': '<|im_start|>system\nGenerate the POSCAR file for the given crystal structure.<|im_end|>\n<|im_start|>user\nLiGa(SeO₃)₂ crystallizes in the tetragonal I̅42d space group. Li¹⁺ is bonded in a 4-coordinate geometry to four O²⁻ atoms. There are two shorter (2.01 Å) and two longer (2.03 Å) Li-O bond lengths. Ga³⁺ is bonded in an octahedral geometry to six O²⁻ atoms. There are a spread of Ga-O bond distances ranging from 1.99-2.03 Å. Se⁴⁺ is bonded in a distorted trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.72-1.75 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal non-coplanar geometry to one Li¹⁺, one Ga³⁺, and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a bent 120 degrees geometry to one Ga³⁺ and one Se⁴⁺ atom.<|im_end|>\n<|im_start|>assistant\nLi¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁺, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li¹⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1⁆, one Li1�52, one Li1�52, one Li1�52, one Li1�52, one Li1�52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one Li1^52, one'}]